In [2]:
!pip install azure.datalake.store 


  Using cached azure_datalake_store-0.0.48-py2.py3-none-any.whl (53 kB)
  Using cached adal-1.2.3-py2.py3-none-any.whl (53 kB)


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as sum_
from azure.datalake.store import core, lib, multithread
import pandas as pd
from datetime import timedelta

spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()

adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BD77FM7QT to authenticate.


### (0) Preprocess: Lê os dados do lake

In [6]:
# Lê dados de predição
df = pd.DataFrame()
output_path = 'adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/09_reporting/process_output/'
files = ['diesel_s10/backtest_diesels10.csv', 'diesel_s500/backtest_diesels500.csv', 
         'etanol/backtest_etanol.csv', 'gasolina/backtest_gasolina.csv']
for f in files:
    tmp = spark.read.format("csv").option("header", "true").load(output_path + f).toPandas()
    df = pd.concat([df, tmp])
    del tmp

# Lê dados de realizado
realizado = spark.read.format("csv").option("header", "true").load(output_path + './../realizado/realizado.csv').toPandas()

AnalysisException: 'Path does not exist: adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/09_reporting/process_output/diesel_s10/backtest_diesels10.csv;'

### (1) Preprocess: substitui colunas de Cidade e UF por código IBGE

In [2]:
'''
Lê base de referência das cidades
'''

# Lê tabela com códigos de cidades e inclui na tabela principal, checa se não faltou nada, exclui colunas de cidade e uf
cod_cidades = pd.read_csv('/home/jovyan/project/projecao-demanda/data/03_primary/cidade_ibge.txt', sep=';', encoding='latin1')
cod_cidades = cod_cidades[['nome_cidade', 'sigla_uf', 'codigo_ibge']]
cod_cidades.columns = ['Cidade', 'UF', 'cod_cidade']

In [ ]:
'''
Ajuste com base de predição
'''

# Inclui na tabela principal
df = df.merge(cod_cidades, how='left')

# Checa se não faltou nada
cidades_sem_codigo = df[['Cidade', 'UF']].drop_duplicates()
cidades_com_codigo = df[['Cidade', 'UF', 'cod_cidade']].drop_duplicates()
(cidades_sem_codigo == cidades_com_codigo[['Cidade', 'UF']]).sum()/cidades_sem_codigo.shape[0]

# exclui colunas de cidade e uf
df = df.drop(['Base', 'Ano', 'Mes', 'Cidade', 'UF'], axis=1)

# reformata tipos do dataframe
df.Date = pd.to_datetime(df.Date)
df.cutoff = pd.to_datetime(df.cutoff)
df.Volume = df.Volume.astype('float32')

# grava pickle pra facilitar se precisar restagar depois
df.to_pickle('/home/jovyan/project/projecao-demanda/data/03_primary/backtest.pkl')

In [3]:
'''
Ajuste com base de realizado
'''

# Inclui na tabela principal
realizado = realizado.merge(cod_cidades, how='left')
realizado.Date = pd.to_datetime(realizado.Date)
realizado.Mes = realizado.Mes.astype('int32')
realizado.Ano = realizado.Ano.astype('int32')
realizado.Volume = realizado.Volume.astype('float32')

# Checa se não faltou nada
cidades_sem_codigo_realizado = realizado[['Cidade', 'UF']].drop_duplicates()
cidades_com_codigo_realizado = realizado[['Cidade', 'UF', 'cod_cidade']].drop_duplicates()
(cidades_sem_codigo_realizado == cidades_com_codigo_realizado[['Cidade', 'UF']]).sum()/cidades_sem_codigo_realizado.shape[0]

# exclui colunas de cidade e uf
realizado = realizado.drop(['Cidade', 'UF'], axis=1)

# grava pickle pra facilitar se precisar restagar depois
realizado.to_pickle('/home/jovyan/project/projecao-demanda/data/03_primary/realizado.pkl')

NameError: name 'realizado' is not defined

### (2) Rateio Cidade x Base

In [4]:
# Lê arquivos atualizados de predição (df) e realizado
df = pd.read_pickle('/home/jovyan/project/projecao-demanda/data/03_primary/backtest.pkl')
realizado = pd.read_pickle('/home/jovyan/project/projecao-demanda/data/03_primary/realizado.pkl')

# Cria visualização com cutoff com lag de 28 dias, para que seja utilizado um rateio com dados históricos por base
df['lag_cutoff'] = df.cutoff + timedelta(-28)
df['Ano'] = df.lag_cutoff.dt.year
df['Mes'] = df.lag_cutoff.dt.month

In [5]:
# Cria tabela com definição de rateio histórico
realizado_rateio_total = realizado\
                                .groupby(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'])['Volume']\
                                .sum()\
                                .reset_index()

realizado_rateio_base = realizado\
                                .groupby(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto', 'Base'])['Volume']\
                                .sum()\
                                .reset_index()

realizado_rateio_base = realizado_rateio_base.merge(
    realizado_rateio_total, 
    on=['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'],
    suffixes=('_Base', '_Total'))
realizado_rateio_base['Volume_Prop'] = realizado_rateio_base.Volume_Base / realizado_rateio_base.Volume_Total

realizado_rateio_base.loc[realizado_rateio_base.Volume_Prop.isnull(),'Volume_Prop'] = 1.0

# Inclui coluna com data do rateio
realizado_rateio_base['Date_Rateio'] = pd.to_datetime(
    realizado_rateio_base['Ano'].astype(str)+ realizado_rateio_base['Mes'].astype(str), 
    format='%Y%m')

# Inclui descritivo da cidade
realizado_rateio_base = realizado_rateio_base.merge(cod_cidades)

#### (2.1) Primeira Tentativa: Rateio com histórico de 28 dias, chave Ano\Mes\cod_cidade\Segmento\Produto

In [6]:
pool = df[['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto']].drop_duplicates()
print('Devem ser preenchidas ', pool.shape[0], ' combinações de Ano/Mes/cod_cidade/Segmento/Produto.')

Devem ser preenchidas  187996  combinações de Ano/Mes/cod_cidade/Segmento/Produto.


In [7]:
'''
Realiza o preenchimento daquelas chaves (Ano/Mes/cod_cidade/Segmento/Produto) em que houve um realizado no mês
anterior, e que podemos utilizar a distribuição das bases
'''

df1 = df.merge(
    realizado_rateio_base[['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto', 'Base', 'Volume_Prop']], 
    how='left'
)
df1 = df1[~(df1.Base.isnull())]
df1 = df1[['Date', 'cutoff', 'contexto', 'Segmento', 'Produto', 'Volume',
       'cod_cidade', 'lag_cutoff', 'Ano', 'Mes', 'Base', 'Volume_Prop']]

pool_df1 = df1\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume_Prop'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')

In [8]:
print('Ainda nulls: ', pool.shape[0] - pool_df1.shape[0])
print('Percentual: ', (pool.shape[0] - pool_df1.shape[0]) / pool.shape[0])

Ainda nulls:  74866
Percentual:  0.39823187727398457


#### (2.2) Segunda Tentativa: Incluir histórico mais recente preenchido

In [9]:
# Lista a parte do DataFrame que falta realizar rateio
pool_faltantes = pool[(pool.merge(pool_df1, how='outer', indicator=True)._merge=='left_only').values]
df2 = df.merge(pool_faltantes, how='right')
try:
    df2 = df2.drop(['Base', 'Volume_Prop'], axis=1)
except:
    pass

In [10]:
'''
Realiza o preenchimento daquelas chaves (Ano/Mes/cod_cidade/Segmento/Produto) em que não houve um realizado no mês
anterior, então busca o último período em que houve um realizado no histórico da chave
'''

# Inclui informações de rateio de realizado com a defasagem de data específica
df2 = df2.merge(
    realizado_rateio_base[['Date_Rateio', 'cod_cidade', 'Segmento', 'Produto', 'Base', 'Volume_Prop']],
    on = ['cod_cidade', 'Segmento', 'Produto'],
    how='left'
)
df2['Rateio_Delta'] = (df2.Date - df2.Date_Rateio).dt.days

# Mantém apenas linhas que possuem algum realizado posterior (e coloca o mais recente)
df2.loc[(df2.Rateio_Delta<0.0), ['Date_Rateio', 'Base', 'Volume_Prop', 'Rateio_Delta']] = \
    [df2.iloc[0].Date_Rateio, df2.iloc[0].Base, df2.iloc[0].Volume_Prop, df2.iloc[0].Rateio_Delta]
df2 = df2.merge(
    df2[~(df2.Date_Rateio.isnull())].groupby(['cod_cidade', 'Produto', 'Segmento'])['Date_Rateio'].min().reset_index(),
    how='right'
)

# Organiza base
df2 = df2[~(df2.Base.isnull())]
df2 = df2[['Date', 'cutoff', 'contexto', 'Segmento', 'Produto', 'Volume',
       'cod_cidade', 'lag_cutoff', 'Ano', 'Mes', 'Base', 'Volume_Prop']]

# Lista casos encontrados
pool_df2 = df2\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume_Prop'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')


In [11]:
print('Ainda nulls: ', pool.shape[0] - pool_df1.shape[0] - pool_df2.shape[0])
print('Percentual: ', (pool.shape[0] - pool_df1.shape[0] - pool_df2.shape[0]) / pool.shape[0])

Ainda nulls:  46958
Percentual:  0.24978191025340965


#### (2.3) Terceira tentativa: Lista a base que melhor atendeu aquele estado para o mesmo produto/segmento no ano-mes de referência

In [12]:
# Lista a parte do DataFrame que falta realizar rateio
pool_realizados = pd.concat([pool_df1, pool_df2], ignore_index=True)
pool_faltantes = pool[(pool.merge(pool_realizados, how='outer', indicator=True)._merge=='left_only').values]
df3 = df.merge(pool_faltantes, how='right')
try:
    df3 = df3.drop(['Base', 'Volume_Prop'], axis=1)
except:
    pass

In [ ]:
'''
Realiza o preenchimento daquelas chaves (Ano/Mes/cod_cidade/Segmento/Produto) em que não houve um realizado em nenhum
ponto no histórico da chave, portanto 
'''

In [13]:
# Cria base com realizado por UF e Base
realizado_UF_base = realizado_rateio_base\
.groupby(['Ano', 'Mes', 'UF', 'Produto', 'Segmento', 'Base'])['Volume_Base']\
.sum()\
.reset_index()

# Cria base com realizado por UF
realizado_UF = realizado_rateio_base\
.groupby(['Ano', 'Mes', 'UF', 'Produto', 'Segmento'])['Volume_Base']\
.sum()\
.reset_index()
realizado_UF.columns = ['Ano', 'Mes', 'UF', 'Produto', 'Segmento', 'Volume_UF']

# Define proporcionalidade de Base por UF
realizado_UF_base = realizado_UF_base.merge(realizado_UF, how='left')
realizado_UF_base['Volume_Prop'] = realizado_UF_base.Volume_Base / realizado_UF_base.Volume_UF

# Inclui coluna com data do rateio
realizado_UF_base['Date_Rateio'] = pd.to_datetime(
    realizado_UF_base['Ano'].astype(str)+ realizado_UF_base['Mes'].astype(str), 
    format='%Y%m')

In [ ]:
# Inclui dados de proporcionalidade de Base por UF nos registros faltantes
df3 = df3.merge(cod_cidades, how='left')
df3 = df3.merge(
    realizado_UF_base[['UF', 'Segmento', 'Produto', 'Base', 'Volume_Prop', 'Date_Rateio']], 
    how='left'
)

In [ ]:
df3

In [ ]:
# Organiza base
df3 = df3[~(df3.Base.isnull())]
df3 = df3[['Date', 'cutoff', 'contexto', 'Segmento', 'Produto', 'Volume',
       'cod_cidade', 'lag_cutoff', 'Ano', 'Mes', 'Base', 'Volume_Prop']]

# Lista casos encontrados
pool_df3 = df3\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume_Prop'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')

In [ ]:
Date_Rateio

In [ ]:
# Cria base com realizado por UF e Base
realizado_UF_base = realizado_rateio_base\
.groupby(['Ano', 'Mes', 'UF', 'Produto', 'Segmento', 'Base'])['Volume_Base']\
.sum()\
.reset_index()

# Cria base com realizado por UF
realizado_UF = realizado_rateio_base\
.groupby(['Ano', 'Mes', 'UF', 'Produto', 'Segmento'])['Volume_Base']\
.sum()\
.reset_index()
realizado_UF.columns = ['Ano', 'Mes', 'UF', 'Produto', 'Segmento', 'Volume_UF']

# Define proporcionalidade de Base por UF
realizado_UF_base = realizado_UF_base.merge(realizado_UF, how='left')
realizado_UF_base['Volume_Prop'] = realizado_UF_base.Volume_Base / realizado_UF_base.Volume_UF

# Inclui dados de proporcionalidade de Base por UF nos registros faltantes
df3 = df3.merge(cod_cidades, how='left')
df3 = df3.merge(
    realizado_UF_base[['Ano', 'Mes', 'UF', 'Segmento', 'Produto', 'Base', 'Volume_Prop']], 
    how='left'
)

# Organiza base
df3 = df3[~(df3.Base.isnull())]
df3 = df3[['Date', 'cutoff', 'contexto', 'Segmento', 'Produto', 'Volume',
       'cod_cidade', 'lag_cutoff', 'Ano', 'Mes', 'Base', 'Volume_Prop']]

# Lista casos encontrados
pool_df3 = df3\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume_Prop'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')

In [ ]:
print('Ainda nulls: ', pool.shape[0] - pool_df1.shape[0] - pool_df2.shape[0] - pool_df3.shape[0])
print('Percentual: ', (pool.shape[0] - pool_df1.shape[0] - pool_df2.shape[0] - pool_df3.shape[0]) / pool.shape[0])

In [ ]:
pool_df2 = df2\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume_Prop'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')

In [ ]:
pool_df2.shape

In [ ]:
df[(df.cod_cidade==1100064) & (df.Date=='2019-04-28')]

In [ ]:
pool_df3.merge(cod_cidades)

In [ ]:
(pool_df3.Volume_Prop.isnull().sum()) / pool.shape[0]

In [ ]:
pool_df3.Volume_Prop.isnull().sum()

In [ ]:
pool_df3

In [ ]:
df3[(~df3.Base.isnull()) & (df3.Rateio_Delta<0.0)]

In [ ]:
df2.head()

In [ ]:
realizado_rateio_base['Date_Rateio'] = pd.to_datetime(
    realizado_rateio_base['Ano'].astype(str)+ realizado_rateio_base['Mes'].astype(str), 
    format='%Y%m')

first = realizado_rateio_base.merge(
    realizado_rateio_base.groupby(['cod_cidade', 'Produto', 'Segmento'])['Date_Rateio'].min().reset_index(),
    how='right'
)

In [ ]:
pd.to_datetime(realizado_rateio_base[['Ano', 'Mes', 'Dia']])

In [ ]:
realizado_rateio_base[['cod_cidade', 'Produto', 'Segmento']]

In [ ]:
df1[~df1.Base.isnull()]

In [ ]:
pool

In [ ]:
df\
.sort_values(['Date', 'cod_cidade', 'Segmento', 'Produto', 'Volume'])\
.drop_duplicates(['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto'], keep='last')\
.Volume.isnull().sum()

In [ ]:
realizado_rateio_base

In [ ]:
realizado_rateio_base

In [ ]:
realizado_rateio_base.columns

In [ ]:
realizado_rateio_base.dtypes

In [ ]:
df = df.merge(
    realizado_rateio_base[['Ano', 'Mes', 'cod_cidade', 'Segmento', 'Produto', 'Base', 'Volume_Prop']], 
    how='outer'
)

In [ ]:
df

In [ ]:
df = df.merge(unique_realizado, how='outer')

In [ ]:
df.shape

In [ ]:
df[(df.cod_cidade==4208906) & (df.Produto=='Diesel S10') & (df.Segmento=='B2B')]

In [ ]:
df.tail()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
result_pdf = df.select("*").toPandas()